In [3]:
import os
import shutil
from sklearn.model_selection import train_test_split
import glob

# Flowers dataset path on Kaggle
KAGGLE_INPUT = "/kaggle/input/flowers-recognition/flowers"

# Create directory structure
os.makedirs("data/train", exist_ok=True)
os.makedirs("data/val", exist_ok=True)

# Get all flower classes
classes = [d for d in os.listdir(KAGGLE_INPUT) 
           if os.path.isdir(os.path.join(KAGGLE_INPUT, d))]

for cls in classes:
    src_folder = os.path.join(KAGGLE_INPUT, cls)
    
    os.makedirs(f"data/train/{cls}", exist_ok=True)
    os.makedirs(f"data/val/{cls}", exist_ok=True)
    
    # Get all images and split 80/20
    images = glob.glob(f"{src_folder}/*")
    train_imgs, val_imgs = train_test_split(
        images, test_size=0.2, random_state=42
    )
    
    for img in train_imgs:
        shutil.copy(img, f"data/train/{cls}/")
    for img in val_imgs:
        shutil.copy(img, f"data/val/{cls}/")

print("✅ Dataset ready!")
print(f"Classes: {classes}")
print(f"Train images: {sum(len(os.listdir(f'data/train/{c}')) for c in classes)}")
print(f"Val images: {sum(len(os.listdir(f'data/val/{c}')) for c in classes)}")

✅ Dataset ready!
Classes: ['dandelion', 'daisy', 'sunflower', 'tulip', 'rose']
Train images: 3452
Val images: 865


In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
img_size = (224, 224)
num_classes = 5  # your dataset
# Data pipeline (example – adapt to your folders)
train_ds = keras.preprocessing.image_dataset_from_directory(
    "data/train",
    image_size=img_size,
    batch_size=32
)
val_ds = keras.preprocessing.image_dataset_from_directory(
    "data/val",
    image_size=img_size,
    batch_size=32
)
# Data augmentation
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])
# Stage 1: freeze base
base_model = keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=img_size + (3,)
)
base_model.trainable = False
inputs = keras.Input(shape=img_size + (3,))
x = data_augmentation(inputs)
x = keras.applications.resnet50.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation="relu")(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
model.fit(train_ds, epochs=5, validation_data=val_ds)
# Stage 2: unfreeze top block 
base_model.trainable = True
for layer in base_model.layers[:-30]:
    layer.trainable = False
model.compile(
    optimizer=keras.optimizers.Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
model.fit(train_ds, epochs=5, validation_data=val_ds)

Found 3452 files belonging to 5 classes.
Found 865 files belonging to 5 classes.
Epoch 1/5
108/108 ━━━━━━━━━━━━━━━━━━━━ 28s 203ms/step - accuracy: 0.7500 - loss: 0.7233 - val_accuracy: 0.8902 - val_loss: 0.3044
Epoch 2/5
108/108 ━━━━━━━━━━━━━━━━━━━━ 19s 175ms/step - accuracy: 0.9057 - loss: 0.2600 - val_accuracy: 0.8925 - val_loss: 0.3365
Epoch 3/5
108/108 ━━━━━━━━━━━━━━━━━━━━ 18s 169ms/step - accuracy: 0.9256 - loss: 0.1974 - val_accuracy: 0.9133 - val_loss: 0.2941
Epoch 4/5
108/108 ━━━━━━━━━━━━━━━━━━━━ 18s 169ms/step - accuracy: 0.9378 - loss: 0.1749 - val_accuracy: 0.8971 - val_loss: 0.3212
Epoch 5/5
108/108 ━━━━━━━━━━━━━━━━━━━━ 19s 172ms/step - accuracy: 0.9563 - loss: 0.1124 - val_accuracy: 0.9052 - val_loss: 0.3170
Epoch 1/5
108/108 ━━━━━━━━━━━━━━━━━━━━ 38s 252ms/step - accuracy: 0.9336 - loss: 0.1927 - val_accuracy: 0.9040 - val_loss: 0.4832
Epoch 2/5
108/108 ━━━━━━━━━━━━━━━━━━━━ 25s 228ms/step - accuracy: 0.9760 - loss: 0.0841 - val_accuracy: 0.9237 - val_loss: 0.3156
Epoch 3/5